In [1]:
import numpy as np
from sklearn import datasets
from sklearn import preprocessing
from sklearn import model_selection

In [2]:
features, targets = datasets.load_breast_cancer(return_X_y=True)

In [3]:
features.shape

(569, 30)

In [4]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(features, targets, test_size=0.2, random_state=0)

In [5]:
n_features = x_train.shape[-1]
n_features

30

In [6]:
# Input features are centered and scaled to unit variance:
feature_encoder = preprocessing.StandardScaler()
x_train = feature_encoder.fit_transform(x_train)
x_test = feature_encoder.transform(x_test)

In [7]:
x_train.shape[0], x_test.shape[0]

(455, 114)

In [8]:
# number of neurons per layer, the last element must be 1
n_neurons = np.array([100, 10, 1])
n_branches = 20  # number of dendritic brancher per neuron

In [9]:
n_neurons

array([100,  10,   1])

---

In [10]:
import torch.nn as nn
import torch

In [135]:
class DentricLayer(nn.Module):
    def __init__(self, 
                 n_input:int,
                 n_output:int, 
                 n_branches:int,
                 hyperplane_bias_magnitude:float = 1.):
        
        super(DentricLayer, self).__init__()
        
        self.dgn_weights = nn.Parameter(
            torch.zeros(n_output, n_branches, n_input + 1),
            requires_grad=False,
        )
        self.dgn_hyperplanes = torch.normal(
            0, 1, [n_output, n_branches, n_input + 1],
            requires_grad=False,
        )
        norm = torch.linalg.norm(self.dgn_hyperplanes[:, :, 1:], axis=(1, 2))[:, None, None]
        self.dgn_hyperplanes = self.dgn_hyperplanes / norm
        
        self.hyperplane_bias_magnitude = torch.FloatTensor([hyperplane_bias_magnitude])
        
    def forward(self, X):
        x_in = torch.hstack([X, torch.ones((X.shape[0], 1))])
        filtered_x = torch.hstack([X, torch.ones((X.shape[0], 1)) * self.hyperplane_bias_magnitude])
        
        gate_values = torch.matmul(self.dgn_hyperplanes, filtered_x.T)
        gate_values = torch.heaviside(gate_values, torch.tensor(0.))
        
        effective_weights = torch.tensordot(gate_values, self.dgn_weights, dims=([1], [1])).sum(dim=0)
        r_out = torch.einsum('ijk,ki->ij', effective_weights, x_in.T)
        
        return r_out

In [137]:
class DGNModel(nn.Module):
    def __init__(self):
        super(DGNModel, self).__init__()
        
        self.l1 = DentricLayer(n_input=31,
                  n_branches=20, 
                  n_output=100)
        
        self.l2 = DentricLayer(n_input=100,
                  n_branches=20, 
                  n_output=10)
        
        self.l3 = DentricLayer(n_input=10,
                  n_branches=20, 
                  n_output=1)
        
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [138]:
# model = nn.Sequential(
#     DentricLayer(n_input=31,
#     n_branches=20, 
#     n_output=100),
    
#     DentricLayer(n_input=100,
#     n_branches=20, 
#     n_output=10),
    
#     DentricLayer(n_input=10,
#     n_branches=20, 
#     n_output=1)
# )

In [139]:
model = DGNModel()

In [140]:
for l in model.parameters():
    print(l)

Parameter containing:
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],


In [121]:
a = list(model.parameters())

In [111]:
y_hat = model(x)

In [112]:
y_hat.shape

torch.Size([455, 1])

[link](https://stackoverflow.com/questions/76113195/custom-optimizer-in-pytorch-or-tensorflow-2-12-0)

In [153]:
def loss(r, target):
    return r - target

class optimizer(torch.optim.Optimizer):
    def __init__(self, params, lr=0.001):
        defaults = dict(lr=lr)
        super(optimizer, self).__init__(params, defaults)
        self.lr = {}

        for group in self.param_groups:
            for param in group['params']:
                self.lr[param] = torch.ones_like(param.data) * lr
    
    def step(self):
        pass

In [141]:
model.parameters()

<generator object Module.parameters at 0x0000019E346684A0>

In [88]:
for param in model.parameters():
    print(param)

In [114]:
L1 = DentricLayer(n_input=31,
                  n_branches=20, 
                  n_output=100)

In [143]:
for name, param in model.named_parameters():
    print(f"Name: {name}")

Name: l1.dgn_weights
Name: l2.dgn_weights
Name: l3.dgn_weights


In [148]:
model._modules.get('l1')(x)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [36]:
x = np.hstack([np.ones((x_train.shape[0], 1)), x_train])
x = torch.FloatTensor(x)
L1(x)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])